In [ ]:
import pickle
import numpy as np
import os
import sys
sys.path.insert(0, os.getcwd()[:-6])
from v2i.src.core.common import loadPKL, savePKL

## Moving from time one time period to other

Earlier we were setting time period of $t=0.1$, which was too small. Based upon some calculation given here, we figured out $t=0.3$ will be good sampling rate to simulate. But how to change tfTrajectories files to make it work/sense with different time period

### Copy Dictionary Structure

Copy the dictionary structre from existing tfTrajectory file. Provide the file path here to clone

In [2]:
# dictionary structure to copy
file = "/home/mayank/Documents/upgraded-octo-lamp/v2i/src/data/tftrajec.pkl"

### Have a look at directory struicture. 
It has three components.
    1. metadata - it maintains the information about the dictionary which is used by simulator to validate details.
    2. data - list of pts
    3. numTrajecs - size of data list

In [3]:
d = loadPKL(file)

In [4]:
d.keys()

dict_keys(['metadata', 'data', 'numTrajecs'])

In [5]:
d['metadata']

{'max': 60,
 'min': 30,
 'time-period': 0.3,
 'num-tries': 200,
 'horizon': 10000,
 'pts': 10}

### Reset the data list and trajectories counter
We will generate the new data and will increment the counter. 

In [6]:
d['data'] = []
print(d['data'])

[]


In [7]:
d['numTrajecs'] = 0
print(d['numTrajecs'])

0


# Calculate new max/min duration of traffic lights

min duration of traffic light is calculated based on the time agent takes to reach to traffic light if it is travelling at max speed of 10m/s. In similar way, we can convert those seconds into steps

In [8]:
minTime = 30
maxTime = 60
minSteps = 100
maxSteps = 200
horizon = 3350

In [9]:
def randomTimeDuration(minDur, maxDur, num):
    durs = np.random.randint(minDur, maxDur, num)
    return durs
        

In [10]:
allPts = [[500, 1200, 1600, 2300, 2800, 3100] ,
          [200, 700, 1000, 1500, 1900, 2500],
          [100, 400, 1400, 2000, 2500, 2700],
          [100, 1000, 1500, 2200, 2800, 3100],
          [300, 800, 1400, 1700, 2000, 2700],
          [1000, 1600, 2100, 2500, 2900, 3200],
          [1200, 1600, 2100, 2500, 2900, 3200]]

# Generate points

In [11]:
d['data'] = []
d['numTrajecs'] = 0
for pts in allPts:
    durs = randomTimeDuration(minSteps, maxSteps, len(pts))
    newList = []
    for index, p in enumerate(pts):
        newList.append([p, durs[index]])
    d['data'].append(newList)
    d['numTrajecs'] += 1
print(d['data'])

[[[500, 129], [1200, 198], [1600, 187], [2300, 178], [2800, 191], [3100, 140]], [[200, 135], [700, 140], [1000, 165], [1500, 139], [1900, 163], [2500, 144]], [[100, 172], [400, 147], [1400, 117], [2000, 118], [2500, 118], [2700, 173]], [[100, 169], [1000, 172], [1500, 100], [2200, 161], [2800, 152], [3100, 141]], [[300, 169], [800, 111], [1400, 153], [1700, 140], [2000, 197], [2700, 117]], [[1000, 183], [1600, 118], [2100, 109], [2500, 110], [2900, 190], [3200, 192]], [[1200, 189], [1600, 142], [2100, 156], [2500, 112], [2900, 169], [3200, 133]]]


In [12]:
d['metadata']['pts'] = len(d['data'][0])
d['metadata']['horizon'] = horizon

### Validate generated pts


In [13]:
for pts in d["data"]:
    for p in pts:
        low = p[0]
        high = p[0] + p[1]
        diff = high - low
        
        if diff >= minSteps and diff <= maxSteps:
            pass
        else:
            raise ValueError("invalid @ ", p, "Difference : ", diff)
        

### Save Final Dictionary

In [15]:
savePKL(d, file)

True